### Python lubi sekwencje

In [1]:
import collections

Card = collections.namedtuple("Card", ("rank", "suit"))


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list("JQKA")
    suits = "spades diamonds clubs hearts".split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [2]:
deck = FrenchDeck()

print(Card('2', 'spades') in deck)
for card in deck[:5]:
    print(card)

True
Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')


Poprawna podklasa `Sequence` musi implementować `__getitem__` i `__len__`. Wszystkie inne metody w `Sequence` są konkretne więc podklasy mogą je dziedziczyć. Zauważmy, że nie ma metody `__iter__` a jednak `FrenchDeck` można iterować, ponieważ gdy Python natrafia na metodę `__getitem__`, próbuje iterować po obiekcie, wywołując tę metodę z indeksami całkowitymi. Ponieważ Python radzi sobie z iterowaniem, to można też korzystać z operatora `in`, mimo że brakuje metody `__contains__`, która wykonuje sekwencyjne skanowanie. Zachowania te są implementowane w samym interpreterze, głównie w języku C.

### Małpie łatanie

Małpie łątanie to dynamiczne zmienianie modułu, klasy lub funkcji w trakcie wykonywania programu, aby dodawać nowe funkcje lub naprawiać błędy. Przykładowo zastosujmy małpie łatanie dla klasy `FrenchDeck` tak aby zapewnić możliwość tasowania (obecnie nie jest to możliwe ze względu na brak metody `__setitem__`).

In [3]:
import random

deck = FrenchDeck()
random.shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [4]:
def set_card(deck, position, card):
    deck._cards[position] = card


FrenchDeck.__setitem__ = set_card
random.shuffle(deck)
deck[:5]

[Card(rank='Q', suit='spades'),
 Card(rank='J', suit='spades'),
 Card(rank='9', suit='clubs'),
 Card(rank='6', suit='spades'),
 Card(rank='8', suit='spades')]

### Programowanie defensywne - szybkie porażki

In [5]:
# Kacze typowanie do obsługi łańcucha znaków lub iterowalnego obiektu z łańcuchami znaków.
field_names = ["card", "suit"]

try:
    field_names = field_names.replace(",", " ").split()
except:
    pass

field_names = tuple(field_names)
if not all(s.isidentifier() for s in field_names):
    raise TypeError("field_names names must be valid identifiers")

### Gęsie typowanie

Gęsie typowanie obejmuje:
- Tworzenie podklas z abstrakcyjnej klasy bazowej aby było jasne, że implementujemy wcześniej zdefiniowany interfejs.
- Sprawdzanie typów w czasie wykonywania programu przy użyciu klas ABC.

In [6]:
from collections import namedtuple, abc

Card = namedtuple("Card", ["rank", "suit"])


class FrenchDeck(abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list("JQKA")
    suits = "spades diamonds clubs hearts".split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    # Sekwencje muszą implementować __len__ i __getitem__
    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

    # MutableSequence wymaga implementacji __setitem__, __delitem__, insert
    # mimo, że nie będziemy ich używać.
    def __setitem__(self, position, card):
        self._cards[position] = card

    def __delitem__(self, position):
        del self._cards[position]

    def insert(self, position, card):
        self._cards.insert(position, card)

### Tworzenie i wykorzystywanie własnej abstrakcyjnej klasy bazowej

Abstrakcyjne klasy bazowe, podobnie jak deskryptory i metaklasy, to narzędzia do budowania platform programistycznych. Jedynie niewielka (< 0.1%) część programistów Pythona będzie tworzyć klasy ABC, bez wprowadzania niepotrzebnych ograniczeń.

In [7]:
import abc

from abc import ABCMeta


class Tombola(abc.ABC):
    @abc.abstractmethod
    def load(self, iterable):
        """Dodaje elementy z iterable"""

    @abc.abstractmethod
    def pick(self):
        """Losuje element, usuwa go z bębna i zwraca"""

    def loaded(self):
        """Zwraca True jeśli w bębnie znajdują się jakieś elementy"""
        return bool(self.inspect())

    def inspect(self):
        """Zwraca tuple z elementami w bębnie"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(items)

Kod dla metody `inspect()` nie jest zbyt sensowny ale pokazuje, że polegając na `pick()` oraz `load()` możemy zbadać co jest wewnątrz obiektu `Tombola`, wybierając te elementy i ładując je z powrotem, bez wiedzy na temat tego jak są przechowywane. Celem tego jest pokazanie, że w klasach ABC mogą być implementowane konkretne metody o ile zależą one tylko od innych metod w interfejsie.

In [8]:
class Fake(Tombola):
    def __init__(self):
        pass

    def pick(self):
        return 0

In [9]:
# Nie możemy utworzyć instancji klasy
# jeśli nie implementuje ona metod abstrakcyjnych z klasy ABC
fake = Fake() 

TypeError: Can't instantiate abstract class Fake with abstract method load

In [10]:
class MyABC(abc.ABC):
    @classmethod
    @abc.abstractmethod
    def an_abstract_classmethod(cls, *args, **kwargs):
        pass

Możemy łączyć dekoratory w interfejsach ale pod warunkiem, że dekorator `abc.abstractmethod` jest stosowany jako najbardziej wewnętrzny dekorator, tzn.  pomiędzy sygnaturą funkcji a dekoratorem `abc.abstractmethod` nie znajduje się nic innego.

In [11]:
class BingoCage(Tombola):
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)

    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError("pick from empty BingoCage")

    def __call__(self):
        return self.pick()

In [12]:
bingo = BingoCage(range(10))
bingo.inspect()

(6, 3, 7, 2, 9, 0, 1, 4, 5, 8)

In [13]:
bingo()

3

In [14]:
class LottoBlower(Tombola):
    def __init__(self, iterable):
        self._balls = list(iterable)  # Programowanie defensywne - szybka porażka

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError("pick from empty LottoBlower")
        return self._balls.pop(position)

    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        return tuple(self._balls)

In [15]:
blower = LottoBlower(range(10))
blower.inspect()

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [16]:
blower.pick()

8

### Wirtualna podklasa abstrakcyjnej klasy bazowej

In [18]:
from random import randrange


@Tombola.register  # Rejestrujemy TomboList jako wirtualną podklasę klasy Tombola
class TomboList(list):
    def pick(self):
        if self:
            position = randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError("pop from empty TomboList")

    load = list.extend

    def loaded(self):
        return bool(self)

    def inspect(self):
        return tuple(self)


# Można też tak
# Tombola.register(TomboList)

In [19]:
issubclass(TomboList, Tombola)

True

In [20]:
isinstance(TomboList(), Tombola)

True

In [21]:
TomboList.__mro__

(__main__.TomboList, list, object)

### Typowanie strukturalne z ABC

Klasy `ABC` są zwykle używane przy typowaniu nominalnym. Gdy klasa `Sub` jawnie dziedziczy po klasie `AnABC` albo jest rejestrowana w `AnABC`, nazwa `AnABC` jest łączona z klasą `Sub`. Właśnie dlatego w czasie wykonywania programu wywołanie `issubclass(AnABC, Sub)` zwraca `True`.

Natomiast typowanie strukturalne patrzy na strukturę publicznego interfejsu obiektu w celu określenia jego typu. Obiekt jest spójny z danym typem jeśli implementuje metody zdefiniowane w tym typie. Dynamiczne i statyczne kacze typowanie stanowią dwa podejścia do typowania strukturalnego.

In [24]:
from collections import abc


class Struggle:
    def __len__(self):
        return 0

In [25]:
isinstance(Struggle(), abc.Sized)

True

In [26]:
issubclass(Struggle, abc.Sized)

True

Okazuje się, że niektóre klasy `ABC` również obsługują typowanie strukturalne. Klasa może być uznana za podklasę klasy `ABC` nawet bez rejestracji. Klasa `Struggle` jest traktowana jako podklasa `abc.Sized`, ponieważ implementuje specjalną metodę klasy o nazwie `__sublcasshook__`. Metoda ta sprawcza czy argument klasy ma atrybut o nazwie `__len__`. Jeśli tak to jest traktowana jako podklasa klasy `abc.Sized`. 

In [34]:
import abc

from abc import ABCMeta


class Sized(metaclass=ABCMeta):
    __slots__ = ()

    @abc.abstractmethod
    def __len__(self):
        return 0

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            if any("__len__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented

### Protokoły statyczne

Bez protokołów statycznych nie ma praktycznego sposobu na dodanie wskazówek do typów dla poniższej funkcji `double()` bez ograniczania jej możliwych zastosowań. Poza tym funkcja `double()` będzie działać nawet z typami z przyszłości o ile implementują metodę `__mul__`.

In [36]:
def double(x):
    return x * 2

print(double(1.5))
print(double("abc"))
print(double([1, 2, 3]))

3.0
abcabc
[1, 2, 3, 1, 2, 3]


Obecnie dzięki `typing.Protocol` możemy poinformować narzędzie do statycznego sprawdzania typów, takiego jak `Mypy`, że typ `double` przyjmuje argument `x`, który obsługuje operację `x * 2`, implementuje metodę `__mul__`.

In [37]:
from typing import TypeVar, Protocol

T = TypeVar("T")


class Repeatable(Protocol):
    def __mul__(self: T, repeats: int) -> T:
        ...

In [38]:
RT = TypeVar("RT", bound=Repeatable)


def double(x: RT) -> RT:
    return x * 2

Zmienna `RT` jest ograniczona przez protokół `Repeatable`. Teraz narzędzie do sprawdzania typów będzie wymagać aby faktyczny typ implementował `Repeatable`. Ponadto narzędzie do sprawdzania typów będzie w stanie weryfikować, że parametr `x` jest obiektem, który może być mnożony przez liczbę całkowitą, a wartość zwracana ma taki sam typ co `x`. Nominalny typ `x` nie jest ważny o ile "kwacze" - implementuje `__mul__`.

### Własny protokół statyczny

In [50]:
from typing import Protocol, runtime_checkable, Any


@runtime_checkable
class RandomPicker(Protocol):
    def pick(self) -> Any:
        ...

In [51]:
import random
from typing import Any, Iterable, TYPE_CHECKING


class SimplePicker:
    def __init__(self, items: Iterable) -> None:
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self) -> Any:
        return self._items.pop()


def test_instance() -> None:
    popper: RandomPicker = SimplePicker([1])
    assert isinstance(popper, RandomPicker)


def test_item_type() -> None:
    items = [1, 2]
    popper = SimplePicker(items)
    item = popper.pick()
    assert item in items
    if TYPE_CHECKING:
        reveal_type(item)
    assert isinstance(item, int)

In [52]:
test_instance()

In [53]:
test_item_type()

In [54]:
# Rozszerzanie protokołu


@runtime_checkable
class LoadableRandomPicker(RandomPicker, Protocol):
    def load(self, items: Iterable) -> None:
        ...